<a href="https://colab.research.google.com/github/TheCaduceus/Direct-Link-to-Google-Drive/blob/main/URL_To_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Direct Link To Google Drive**

**Connect your Google Drive Account:**

In [8]:
#@markdown <br><center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png" height="50" alt="Gdrive-logo' "/></center>
#@markdown <center><h3>Connect to Google Account</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/gdrive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /gdrive


**Google Colab Timeout Preventer! Run this Code to prevent TIMEOUT in Google Colab.Before running any code run this!**

In [ ]:
#@markdown <br><center><img src='https://miro.medium.com/max/2036/1*GW4cqNv-SIQ9BGGhssL1bQ.png' height="100" alt="netdata"/></center>
#@title <center>Time Out Preventer (Advanced) </center></strong>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
#@markdown **Run this code to prevent NOTEBOOK from Timeout**
from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True)
if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button"); 
      btn.click() 
      }
    
    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect"); 
      btn.click() 
      }
    }
    
  setInterval(ClickConnect,60000)
  '''))

**Check from which speed Google Will transfer your Files: (Optional)**

In [ ]:
#@markdown <br><center><img src='https://github.com/TheCaduceus/Google-Drive-Cloner/blob/main/Img/st.jpeg?raw=true' height="50" alt="ST-logo"/></center>
#@markdown <center><h3>Check Transfer Speed!</h3></center><br>
!curl -s https://raw.githubusercontent.com/sivel/speedtest-cli/master/speedtest.py | python

**Single/Multiple URL(s) with Extraction and Combination**

**Before using this! Download Required Setup files and config/update them.**

In [ ]:
#@markdown <br><center><img src='https://github.com/TheCaduceus/Torrent-to-GoogleDrive/blob/main/Img/80.png?raw=true' height="50">
#@markdown <center><h3>Setup Environment and Update Files</h3></center>
!pip install pySmartDL rarfile

**Run the following code and provide One/Multiple Links with Automatic Extraction of ```.zip``` or ```.rar``` | Extraction will happen if the downloading file is in ```.zip``` or ```.rar``` after confirmation from User.**

**Save to My Drive:**

In [ ]:
#@markdown <br><center><img src='https://th.bing.com/th/id/R.e09dea0066e6fc5f0c550fd299ea899a?rik=nMlcMah56a7WJw&riu=http%3a%2f%2ficons.iconarchive.com%2ficons%2fwebalys%2fkameleon.pics%2f512%2fDownload-Computer-icon.png&ehk=HvQcpWGlJteoPtsZDnGBwtkpXvkUk1w%2fVNuPhiS0dBA%3d&risl=&pid=ImgRaw&r=0' height="100">
#@markdown <center><h3><b>Download to My Drive</b></h3></center>
import requests
import sys
import time
import os
import rarfile
import shutil
import zipfile
from google.colab import drive
from io import StringIO
from pySmartDL import SmartDL


def extract_compressed(source, dest):
  if source.endswith(".zip"):
    with zipfile.ZipFile(source, "r") as zip_ref:
      zip_ref.extractall(path)

  else:
    with rarfile.RarFile(source, "r") as rar_ref:
      rar_ref.extractall(path)

  os.remove(source)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

# WIP
def download_file_from_google_drive(id, destination):    
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

downloads = []
path = "/gdrive/My Drive/Downloads/"

if not os.path.exists(path):
    os.makedirs(path)

while True:
    link = input("Enter URL Or Type Exit: ")
    if link.lower() == "exit":
        break
    downloads.append(link)

sys.stdout.write("Downloading\n")

while downloads:

    file_name = (downloads[0].split("/")[-1]).replace("%20"," ")

    obj = SmartDL(downloads, progress_bar=False)
    obj.start(blocking=False)

    while not obj.isFinished():
      if obj.get_status() == "downloading":
        sys.stdout.write(f"\rDownloading {file_name} {obj.get_progress_bar()} {obj.get_progress()*100:.2f}% {obj.get_dl_size(human=True)} {obj.get_speed(human=True)}   ")
        time.sleep(0.5)

      else:
        sys.stdout.write(f"\rCombining {file_name}  ")
        time.sleep(1)

    if obj.isSuccessful():      
      if file_name.endswith(('.zip', '.rar')):
        sys.stdout.write(f"\rExtracting  {file_name} ")
        extract_compressed(obj.get_dest(), path)
      else:
        destination = path + obj.get_dest().split("/")[-1]
        
        if os.path.exists(destination):
          sys.stdout.write("\rRemoving file with the same name on destination")
          os.remove(destination)
        
        shutil.move(obj.get_dest(),path)
          
    else:
        sys.stdout.write("There were some errors:")
        for e in obj.get_errors():
                sys.stdout.write(str(e))

    sys.stdout.write(f"\r\rCompleted {file_name} File size: {obj.get_dl_size(human=True)} Time taken: {obj.get_dl_time(human=True)}   \n")
    del downloads[0]
    
sys.stdout.write("\nAll downloads have been complete")

**Save to Shared Drive:**

**First click on the "Show Code" option and Enter Shared Drive name in which you want to save file, The Code will look like this:<br>```path = "/gdrive/Shared drives/Shared Drive Name/Downloads/" ```**

In [ ]:
#@markdown <br><center><img src='https://th.bing.com/th/id/R.e09dea0066e6fc5f0c550fd299ea899a?rik=nMlcMah56a7WJw&riu=http%3a%2f%2ficons.iconarchive.com%2ficons%2fwebalys%2fkameleon.pics%2f512%2fDownload-Computer-icon.png&ehk=HvQcpWGlJteoPtsZDnGBwtkpXvkUk1w%2fVNuPhiS0dBA%3d&risl=&pid=ImgRaw&r=0' height="100">
#@markdown <center><h3><b>Download to Shared Drive</b></h3></center>
import requests
import sys
import time
import os
import rarfile
import shutil
import zipfile
from google.colab import drive
from io import StringIO
from pySmartDL import SmartDL


def extract_compressed(source, dest):
  if source.endswith(".zip"):
    with zipfile.ZipFile(source, "r") as zip_ref:
      zip_ref.extractall(path)

  else:
    with rarfile.RarFile(source, "r") as rar_ref:
      rar_ref.extractall(path)

  os.remove(source)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

# WIP
def download_file_from_google_drive(id, destination):    
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

downloads = [] #Enter SHARED DRIVE NAME Below 
path = "/gdrive/Shared drives/Shared Drive Name/Downloads/" 

if not os.path.exists(path):
    os.makedirs(path)

while True:
    link = input("Enter URL Or Type Exit: ")
    if link.lower() == "exit":
        break
    downloads.append(link)

sys.stdout.write("Downloading\n")

while downloads:

    file_name = (downloads[0].split("/")[-1]).replace("%20"," ")

    obj = SmartDL(downloads, progress_bar=False)
    obj.start(blocking=False)

    while not obj.isFinished():
      if obj.get_status() == "downloading":
        sys.stdout.write(f"\rDownloading {file_name} {obj.get_progress_bar()} {obj.get_progress()*100:.2f}% {obj.get_dl_size(human=True)} {obj.get_speed(human=True)}   ")
        time.sleep(0.5)

      else:
        sys.stdout.write(f"\rCombining {file_name}  ")
        time.sleep(1)

    if obj.isSuccessful():      
      if file_name.endswith(('.zip', '.rar')):
        sys.stdout.write(f"\rExtracting  {file_name} ")
        extract_compressed(obj.get_dest(), path)
      else:
        destination = path + obj.get_dest().split("/")[-1]
        
        if os.path.exists(destination):
          sys.stdout.write("\rRemoving file with the same name on destination")
          os.remove(destination)
        
        shutil.move(obj.get_dest(),path)
          
    else:
        sys.stdout.write("There were some errors:")
        for e in obj.get_errors():
                sys.stdout.write(str(e))

    sys.stdout.write(f"\r\rCompleted {file_name} File size: {obj.get_dl_size(human=True)} Time taken: {obj.get_dl_time(human=True)}   \n")
    del downloads[0]
    
sys.stdout.write("\nAll downloads have been complete")

**Single URL with Custom Name:
Replace the following:<br> ```URL HERE``` with Direct Download Link, and<br> ```file_name.format``` from custom file name you want to give and also write its original format like (.txt/.mkv/.mp3/.mp4/.pdf/.json/.iso)**

**Save to My Drive:**

In [9]:
import requests
file_url = "URL HERE" #<--Required
	
r = requests.get(file_url, stream = True)
 # No Dependency
with open("/gdrive/My Drive/file_name.format", "wb") as file:
  # Always write format like (.txt/.mp3/.mp4/.pdf/.json/.iso)
	for block in r.iter_content(chunk_size = 1024):
		if block:
			file.write(block)

**Save to Shared Drive:**

**First Replace the Following:<br> ```Shared Drive Name``` with Shared Drive name in which you want to Save the File<br>```URL HERE``` with Direct Download Link, and<br>```file_name.format``` from custom file name you want to give and also write its original format like (.txt/.mkv/.mp3/.mp4/.pdf/.json/.iso)**

In [ ]:
import requests
file_url = "URL HERE" #<--Required
	
r = requests.get(file_url, stream = True)
 # No Dependency
with open("/gdrive/Shared drives/Shared Drive Name/file_name.format", "wb") as file:
  # Always write format like (.txt/.mp3/.mp4/.pdf/.json/.iso)
	for block in r.iter_content(chunk_size = 1024):
		if block:
			file.write(block)